In [1]:
import pandas as pd 

In [2]:
df = pd.read_excel('demo.xlsx')
df.head()

,TIttle_Address,TIttle_level,TIttle_Bedroom,TIttle_Bathroom,TIttle_Toilet,Description
0,Close to Chris EmbassyDurumi Abuja,Premium Gold,2 beds,2 baths,2 Toilets,Specious & Standard 2bedroom with Standby Gen
1,Nicon Town Lekki Lagos,Premium Gold,4 beds,4 baths,5 Toilets,Brand New 4-Bedroom Semi-Detached House with BQ
2,Ajah Lagos,Premium Gold,4 beds,4 baths,5 Toilets,Brand New 4 Bedroom Semi Detached Duplex;
3,Ikate Lekki Lagos,Premium Gold,4 beds,4 baths,5 Toilets,Spacious 4 Bedroom Terraced Duplex
4,Eko Atlantic Victoria Island Lagos,Premium Gold,2 beds,2 baths,3 Toilets,Upscale 2 Bedroom Apartment


In [3]:
#check for missing values 
#iterating the dataset looking for missing value
for i in df.columns:
    print (i + ": "+str(sum(df[i].isnull()))+" missing value")

TIttle_Address: 5 missing value
TIttle_level: 0 missing value
TIttle_Bedroom: 0 missing value
TIttle_Bathroom: 0 missing value
TIttle_Toilet: 0 missing value
Description: 0 missing value


In [4]:
#selecting all missing values
null_data = df[df.isnull().any(axis=1)]
null_data

,TIttle_Address,TIttle_level,TIttle_Bedroom,TIttle_Bathroom,TIttle_Toilet,Description
79,NaN,Premium Gold,3 beds,3 baths,4 Toilets,3 bedroom Apartment
106,NaN,Premium Gold,3 beds,3 baths,4 Toilets,PRINCE ADEMOLA OSINUGA OFFERS!!! LUXURY WATERF...
289,NaN,Premium,10 beds,0 baths,0 Toilets,A Functioning Hotel with 53 Spacious and Well ...
393,NaN,Premium,3 beds,3 baths,4 Toilets,3 bedroom flat
470,NaN,Premium,4 beds,0 baths,0 Toilets,Brand New Exquisitely Finished 4 Bedroom Semi ...


In [5]:
most_frequent_category=df['TIttle_Address'].mode()[0]
df['TIttle_Address'].fillna(most_frequent_category,inplace=True)

* fill missing value with mode for Tittle_level. But inspect it to see if it will make sense

In [6]:
#imorting neccessary library to extract coordinates 
from selenium import webdriver
from tqdm import tqdm_notebook as tqdmn

In [7]:
#remove this line when doing yours 
df = df[0:20]

In [8]:
#looping through google map search address to append each state map search in daaset Eg https://www.google.com/maps/search/LAGOS,https://www.google.com/maps/search/OOYO
df['Url'] = ['https://www.google.com/maps/search/' + i for i in df['TIttle_Address']]

you need to download chrome driver if you use google chrome and place in your system path..call me to explain this step again

In [9]:
#creating an empty list to append the http link that contains each state coordinate
Url_With_Coordinates_df = []

#scraping google map webpage to find the coordinate
option = webdriver.ChromeOptions()
prefs = {'profile.default_content_setting_values': {'images':2, 'javascript':2}}
option.add_experimental_option('prefs', prefs)

#using chromedriver to access the coordinate weblink
driver = webdriver.Chrome("C:\\chromedriver.exe", options=option)

#scraping google map webpage to get the coordinate
for url in tqdmn(df.Url, leave=False):
    driver.get(url)
    Url_With_Coordinates_df.append(driver.find_element_by_css_selector('meta[itemprop=image]').get_attribute('content'))
    
driver.close()

#saving the extracted coordinate to a csv file 
import csv
with open('Url_With_Coordinates_df.csv', 'w') as file:
    wr = csv.writer(file)
    wr.writerow(Url_With_Coordinates_df)

C:\Users\RICHIE\miniconda3\envs\tensorflow1\lib\site-packages\ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


In [10]:
#loading the saved links in urlwithcoordinate file 
import csv
with open('Url_With_Coordinates_df.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    for i in reader:
        Url_With_Coordinates_df = i
        break

In [11]:
#loading 5 links from the saved file 
Url_With_Coordinates_df[:5]

['https://maps.google.com/maps/api/staticmap?center=9.04387725%2C7.4608415&zoom=12&size=200x200&markers=9.0626329%2C7.4674523%7C9.0251216%2C7.4542307&sensor=false&client=google-maps-frontend&signature=cj0fAXfcZE15scHxwcDXs2zWWUc',
 'https://maps.google.com/maps/api/staticmap?center=6.4427302%2C3.4999824&zoom=16&size=256x256&language=en&sensor=false&client=google-maps-frontend&signature=7diC33tk0PPcLnc0qeCn5Np36to',
 'https://maps.google.com/maps/api/staticmap?center=6.46373255%2C3.56818605&zoom=14&size=256x256&language=en&sensor=false&client=google-maps-frontend&signature=OIv5EZX6eNjXoShMGaszA8zb9VY',
 'https://maps.google.com/maps/api/staticmap?center=6.44693775%2C3.52356715&zoom=13&size=256x256&language=en&sensor=false&client=google-maps-frontend&signature=meVelUN17dQzrrAefA6yipzlNsI',
 'https://maps.google.com/maps/api/staticmap?center=6.4097604%2C3.4119852&zoom=17&size=256x256&language=en&sensor=false&client=google-maps-frontend&signature=_s800PfBKaLvSuJQicZWpmA_NMY']

In [12]:
#appending the saved file to our dataset 
df['Url_With_Coordinates_df'] = Url_With_Coordinates_df

In [13]:
#spliting the links to get only the coordinate from each links 
df['lat'] = [url.split('?center=')[1].split('&zoom=')[0].split('%2C')[0] for url in df['Url_With_Coordinates_df'] ]
df['long'] = [url.split('?center=')[1].split('&zoom=')[0].split('%2C')[1] for url in df['Url_With_Coordinates_df'] ]

In [16]:
df.drop(['Url', 'Url_With_Coordinates_df'], axis=1, inplace=True)

In [17]:
df.head()

,TIttle_Address,TIttle_level,TIttle_Bedroom,TIttle_Bathroom,TIttle_Toilet,Description,lat,long
0,Close to Chris EmbassyDurumi Abuja,Premium Gold,2 beds,2 baths,2 Toilets,Specious & Standard 2bedroom with Standby Gen,9.04387725,7.4608415
1,Nicon Town Lekki Lagos,Premium Gold,4 beds,4 baths,5 Toilets,Brand New 4-Bedroom Semi-Detached House with BQ,6.4427302,3.4999824
2,Ajah Lagos,Premium Gold,4 beds,4 baths,5 Toilets,Brand New 4 Bedroom Semi Detached Duplex;,6.46373255,3.56818605
3,Ikate Lekki Lagos,Premium Gold,4 beds,4 baths,5 Toilets,Spacious 4 Bedroom Terraced Duplex,6.44693775,3.52356715
4,Eko Atlantic Victoria Island Lagos,Premium Gold,2 beds,2 baths,3 Toilets,Upscale 2 Bedroom Apartment,6.4097604,3.4119852


In [19]:
#getting other venues from coordinate 
from tqdm import tqdm  # For monitroing the progress
tqdm.pandas()
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='my_app',timeout=None)
from geopy.extra.rate_limiter import RateLimiter

### Extract the Details for Multiple Locations - Data Extraction

In [20]:
def get_suburb(row):
    pos = str(row['lat']) + ', ' + str(row['long'])
    location = geolocator.reverse(pos)
    return location.raw['address']

#### Retrieving the information

In [21]:
df['address'] = df[['lat','long']].progress_apply(get_suburb,axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:14<00:00,  1.39it/s]


In [23]:
df.head()

,TIttle_Address,TIttle_level,TIttle_Bedroom,TIttle_Bathroom,TIttle_Toilet,Description,lat,long,address
0,Close to Chris EmbassyDurumi Abuja,Premium Gold,2 beds,2 baths,2 Toilets,Specious & Standard 2bedroom with Standby Gen,9.04387725,7.4608415,"{'suburb': 'Central Business District', 'city'..."
1,Nicon Town Lekki Lagos,Premium Gold,4 beds,4 baths,5 Toilets,Brand New 4-Bedroom Semi-Detached House with BQ,6.4427302,3.4999824,"{'residential': 'Nicon Town Estate', 'suburb':..."
2,Ajah Lagos,Premium Gold,4 beds,4 baths,5 Toilets,Brand New 4 Bedroom Semi Detached Duplex;,6.46373255,3.56818605,"{'road': 'Emir Road', 'suburb': 'Lekki Phase 2..."
3,Ikate Lekki Lagos,Premium Gold,4 beds,4 baths,5 Toilets,Spacious 4 Bedroom Terraced Duplex,6.44693775,3.52356715,"{'village': 'Ojomu', 'county': 'Eti Osa', 'cou..."
4,Eko Atlantic Victoria Island Lagos,Premium Gold,2 beds,2 baths,3 Toilets,Upscale 2 Bedroom Apartment,6.4097604,3.4119852,"{'village': 'Apese', 'county': 'Eti Osa', 'pos..."


In [24]:
df_Address=df['address'].apply(pd.Series)

In [26]:
df_Address.head(20)

,suburb,city,county,state,postcode,country,country_code,residential,village,road,man_made,neighbourhood
0,Central Business District,Abuja,Municipal Area Council,Federal Capital Territory,223140,Nigeria,ng,NaN,NaN,NaN,NaN,NaN
1,Lekki Phase 1,NaN,Eti Osa,NaN,NaN,Nigeria,ng,Nicon Town Estate,Ikate,NaN,NaN,NaN
2,Lekki Phase 2,NaN,Eti Osa,NaN,NaN,Nigeria,ng,NaN,Ajah,Emir Road,NaN,NaN
3,NaN,NaN,Eti Osa,NaN,NaN,Nigeria,ng,NaN,Ojomu,NaN,NaN,NaN
4,NaN,NaN,Eti Osa,NaN,71510,Nigeria,ng,NaN,Apese,NaN,NaN,NaN
5,Victoria Island,NaN,Eti Osa,NaN,500001,Nigeria,ng,NaN,Apese,Saka Tinubu Street,Lagos State Water Corporation,NaN
6,NaN,NaN,Eti Osa,NaN,20050,Nigeria,ng,NaN,Banana Island,205 Close,NaN,NaN
7,NaN,NaN,Eti Osa,NaN,20050,Nigeria,ng,NaN,Banana Island,205 Close,NaN,NaN
8,Victoria Island,NaN,Eti Osa,NaN,500001,Nigeria,ng,NaN,Apese,Saka Tinubu Street,Lagos State Water Corporation,NaN
9,Lekki Phase 1,NaN,Eti Osa,NaN,500001,Nigeria,ng,NaN,Ogoyo,Lekki-Epe Expressway,NaN,NaN


### Selecting the columns of interest
* This should be done by carefully checking column that has less missing values and having in mind the goal of the task

In [28]:
df_Addressx = df_Address[['road','county','village']]
df_Addressx.head(5)

,road,county,village
0,NaN,Municipal Area Council,NaN
1,NaN,Eti Osa,Ikate
2,Emir Road,Eti Osa,Ajah
3,NaN,Eti Osa,Ojomu
4,NaN,Eti Osa,Apese


In [29]:
df_Merged = df.join(df_Addressx, how='outer')
df_Merged[:10]

,TIttle_Address,TIttle_level,TIttle_Bedroom,TIttle_Bathroom,TIttle_Toilet,Description,lat,long,address,road,county,village
0,Close to Chris EmbassyDurumi Abuja,Premium Gold,2 beds,2 baths,2 Toilets,Specious & Standard 2bedroom with Standby Gen,9.04387725,7.4608415,"{'suburb': 'Central Business District', 'city'...",NaN,Municipal Area Council,NaN
1,Nicon Town Lekki Lagos,Premium Gold,4 beds,4 baths,5 Toilets,Brand New 4-Bedroom Semi-Detached House with BQ,6.4427302,3.4999824,"{'residential': 'Nicon Town Estate', 'suburb':...",NaN,Eti Osa,Ikate
2,Ajah Lagos,Premium Gold,4 beds,4 baths,5 Toilets,Brand New 4 Bedroom Semi Detached Duplex;,6.46373255,3.56818605,"{'road': 'Emir Road', 'suburb': 'Lekki Phase 2...",Emir Road,Eti Osa,Ajah
3,Ikate Lekki Lagos,Premium Gold,4 beds,4 baths,5 Toilets,Spacious 4 Bedroom Terraced Duplex,6.44693775,3.52356715,"{'village': 'Ojomu', 'county': 'Eti Osa', 'cou...",NaN,Eti Osa,Ojomu
4,Eko Atlantic Victoria Island Lagos,Premium Gold,2 beds,2 baths,3 Toilets,Upscale 2 Bedroom Apartment,6.4097604,3.4119852,"{'village': 'Apese', 'county': 'Eti Osa', 'pos...",NaN,Eti Osa,Apese
5,Victoria Island Lagos,Premium Gold,beds,baths,Toilets,Commercial Office Space For Lease,6.42805545,3.4219444,"{'man_made': 'Lagos State Water Corporation', ...",Saka Tinubu Street,Eti Osa,Apese
6,Banana Island Ikoyi Lagos,Premium Gold,4 beds,4 baths,5 Toilets,Premium 4 Bedroom Apartment,6.45947305,3.454854,"{'road': '205 Close', 'village': 'Banana Islan...",205 Close,Eti Osa,Banana Island
7,Banana Island Ikoyi Lagos,Premium Gold,4 beds,4 baths,5 Toilets,Cozy 4 Bedroom Pent House,6.45947305,3.454854,"{'road': '205 Close', 'village': 'Banana Islan...",205 Close,Eti Osa,Banana Island
8,Victoria Island Lagos,Premium Gold,3 beds,3 baths,4 Toilets,Upscale 3 Bedroom Apartment,6.42805545,3.4219444,"{'man_made': 'Lagos State Water Corporation', ...",Saka Tinubu Street,Eti Osa,Apese
9,Ilasan Lekki Lagos,Sponsored,5 beds,5 baths,6 Toilets,4 bedrooms terrace duplex and a maid's room,6.434516,3.493899,"{'road': 'Lekki-Epe Expressway', 'suburb': 'Le...",Lekki-Epe Expressway,Eti Osa,Ogoyo


In [31]:
df_Merged.drop(['address'], axis=1, inplace=True)

In [35]:
df_Merged.tail(5)

,TIttle_Address,TIttle_level,TIttle_Bedroom,TIttle_Bathroom,TIttle_Toilet,Description,lat,long,road,county,village
15,Old Ikoyi Ikoyi Lagos,Premium Gold,3 beds,3 baths,4 Toilets,PRINCE ADEMOLA OSINUGA SPECIALS! ULTRA LUXURY ...,6.44999985,3.433333,Thompson Avenue,Eti Osa,Falomo
16,Ikoyi Lagos,Premium Gold,4 beds,4 baths,5 Toilets,Premium 4 Bedroom Apartment,6.44999985,3.433333,Thompson Avenue,Eti Osa,Falomo
17,...Old Ikoyi Ikoyi Lagos,Premium Gold,5 beds,5 baths,6 Toilets,PRINCE ADEMOLA OSINUGA SPECIALS! LUXURY 5 BEDR...,6.44999985,3.433333,Thompson Avenue,Eti Osa,Falomo
18,Oniru Victoria Island Lagos,Premium Gold,4 beds,4 baths,5 Toilets,Newly Built 4 Bedroom Terraced Duplex,6.4467879,3.4340729,Oniru Road,Eti Osa,Falomo
19,Bourdillon Ikoyi Lagos,Premium Gold,4 beds,5 baths,4 Toilets,Beautiful 4 Bedroom Terrace Duplex with a Bq l...,6.446584,3.4384555,Bourdillon Road,Eti Osa,Falomo


Don't forget to make a shout out to me when you suceed in what you are trying to achieve